In [1]:
import pandas as pd
from src.plots import MovementData

In [2]:
leo = pd.read_csv("data/output/Leo/accelerometer5_data.csv")
matthieu = pd.read_csv("data/output/Matthieu/accelerometer4_data.csv")
serge = pd.read_csv("data/output/Serge/accelerometer4_data.csv")

movement_data = MovementData([leo],["Leo"])

In [3]:
plots = movement_data.make_all_plots()
list(plots.keys())

['Line', 'Violin', 'Stacked', 'XYZ']

In [4]:
plots["Violin"]